In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#from library.sb_utils import save_file
import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing 
import warnings 
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [2]:
bell_2020 = pd.read_csv('../data/bell_cleaned.csv')
bell_sales = pd.read_csv('../data/bell_sales_cleaned.csv')

In [3]:
bell_sales.head()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_rooms,char_beds,char_frpl,char_fbath,char_hbath,char_bldg_sf,geo_longitude,geo_latitude
0,126575,13242020170000,2017.0,278,850000.0,3050.0,18.0,8.0,4.0,0.0,2.0,1.0,2340.0,-87.690565,41.953323
1,126576,13242020180000,2013.0,278,1055000.0,3050.0,2.0,9.0,4.0,3.0,3.0,1.0,2639.0,-87.690473,41.953324
2,126578,13242030020000,2013.0,203,330000.0,3137.0,54.0,6.0,3.0,0.0,2.0,0.0,1020.0,-87.690731,41.952804
3,126579,13242030040000,2014.0,202,300000.0,3050.0,116.0,4.0,2.0,0.0,2.0,0.0,680.0,-87.690547,41.952805
4,126580,13242030110000,2018.0,206,976711.0,3660.0,96.0,7.0,4.0,1.0,2.0,1.0,2778.0,-87.689729,41.952810


In [4]:
bell_sales_pin = bell_sales.meta_pin
bell_y = bell_sales[['meta_sale_price']]
bell_X = bell_sales.drop(['Unnamed: 0','meta_pin','meta_sale_price'],axis=1)

In [5]:
bell_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   meta_year      293 non-null    float64
 1   meta_class     293 non-null    int64  
 2   char_hd_sf     293 non-null    float64
 3   char_age       293 non-null    float64
 4   char_rooms     293 non-null    float64
 5   char_beds      293 non-null    float64
 6   char_frpl      293 non-null    float64
 7   char_fbath     293 non-null    float64
 8   char_hbath     293 non-null    float64
 9   char_bldg_sf   293 non-null    float64
 10  geo_longitude  293 non-null    float64
 11  geo_latitude   293 non-null    float64
dtypes: float64(11), int64(1)
memory usage: 27.6 KB


In [6]:
bell_X = bell_X.astype({'meta_class':'category'})

In [7]:
bell_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   meta_year      293 non-null    float64 
 1   meta_class     293 non-null    category
 2   char_hd_sf     293 non-null    float64 
 3   char_age       293 non-null    float64 
 4   char_rooms     293 non-null    float64 
 5   char_beds      293 non-null    float64 
 6   char_frpl      293 non-null    float64 
 7   char_fbath     293 non-null    float64 
 8   char_hbath     293 non-null    float64 
 9   char_bldg_sf   293 non-null    float64 
 10  geo_longitude  293 non-null    float64 
 11  geo_latitude   293 non-null    float64 
dtypes: category(1), float64(11)
memory usage: 25.9 KB


In [8]:
bell_X = pd.get_dummies(bell_X,prefix='class')

In [9]:
bell_X.head()

,meta_year,char_hd_sf,char_age,char_rooms,char_beds,char_frpl,char_fbath,char_hbath,char_bldg_sf,geo_longitude,geo_latitude,class_202,class_203,class_204,class_205,class_206,class_208,class_278,class_295
0,2017.0,3050.0,18.0,8.0,4.0,0.0,2.0,1.0,2340.0,-87.690565,41.953323,0,0,0,0,0,0,1,0
1,2013.0,3050.0,2.0,9.0,4.0,3.0,3.0,1.0,2639.0,-87.690473,41.953324,0,0,0,0,0,0,1,0
2,2013.0,3137.0,54.0,6.0,3.0,0.0,2.0,0.0,1020.0,-87.690731,41.952804,0,1,0,0,0,0,0,0
3,2014.0,3050.0,116.0,4.0,2.0,0.0,2.0,0.0,680.0,-87.690547,41.952805,1,0,0,0,0,0,0,0
4,2018.0,3660.0,96.0,7.0,4.0,1.0,2.0,1.0,2778.0,-87.689729,41.952810,0,0,0,0,1,0,0,0


for decison trees, keep all dummy columns. For linear regression, drop one.

In [10]:
scaler = StandardScaler()
columns=bell_X.columns
bell_X = pd.DataFrame(scaler.fit_transform(bell_X),columns=columns)
bell_X.head()

,meta_year,char_hd_sf,char_age,char_rooms,char_beds,char_frpl,char_fbath,char_hbath,char_bldg_sf,geo_longitude,geo_latitude,class_202,class_203,class_204,class_205,class_206,class_208,class_278,class_295
0,0.319296,-1.273991,-0.863535,-0.134741,0.065771,-0.681217,-0.438484,0.505319,-0.180187,-1.267246,1.693271,-0.156447,-0.391846,-0.187978,-0.386031,-0.549747,-0.156447,1.200694,-0.117647
1,-1.479816,-1.273991,-1.186774,0.299096,0.065771,2.760104,0.581166,0.505319,0.206375,-1.245920,1.693551,-0.156447,-0.391846,-0.187978,-0.386031,-0.549747,-0.156447,1.200694,-0.117647
2,-1.479816,-1.107048,-0.136246,-1.002414,-1.067812,-0.681217,-0.438484,-1.196502,-1.886750,-1.305810,1.422956,-0.156447,2.552023,-0.187978,-0.386031,-0.549747,-0.156447,-0.832851,-0.117647
3,-1.030038,-1.273991,1.116307,-1.870087,-2.201395,-0.681217,-0.438484,-1.196502,-2.326319,-1.263149,1.423585,6.391959,-0.391846,-0.187978,-0.386031,-0.549747,-0.156447,-0.832851,-0.117647
4,0.769075,-0.103475,0.712257,-0.568578,0.065771,0.465890,-0.438484,0.505319,0.386081,-1.073312,1.426391,-0.156447,-0.391846,-0.187978,-0.386031,1.819017,-0.156447,-0.832851,-0.117647


scaling is needed for models that need to determine a 'distance" between data points. But for models, such as decesion trees, that do not require a distance, it is not required.Scaling should do no harm when it is not needed. 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(bell_X, bell_y, test_size=0.2, random_state=42)

Next step is for all this to go into a pipeline for different models.